<a href="https://colab.research.google.com/github/FatmaBuseBorlu/Dramer-Dataset/blob/leaf/RNN3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.io as sio
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, GaussianNoise, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler  # For balancing classes

In [ ]:
def load_dreamer_data(file_path):
    data = sio.loadmat(file_path)
    eeg_data = []
    labels = []
    for participant in range(23):
        for video in range(18):
            stimuli = data['DREAMER'][0, 0]['Data'][0, participant]['EEG'][0, 0]['stimuli'][0, 0][video, 0]
            if stimuli.size == 0:
                continue
            if stimuli.shape[0] != 14:
                stimuli = stimuli.T
            eeg_data.append(stimuli)

            # Debug: Valence Score Kontrol
            try:
                valence_score = data['DREAMER'][0, 0]['Data'][0, participant]['ScoreValence'][0, 0][video, 0]
                labels.append(valence_score)
            except KeyError:
                print(f"ScoreValence alanı bulunamadı: Participant {participant}, Video {video}")
                continue
    return eeg_data, np.array(labels)


In [ ]:
# Veri Yapısını İnceleme
data = sio.loadmat(file_path)
print("Anahtarlar:", data.keys())
dreamer_data = data['DREAMER']
print("DREAMER Verisinin Tipi:", type(dreamer_data))
print("DREAMER Veri Yapısı:", dreamer_data.dtype)


Anahtarlar: dict_keys(['__header__', '__version__', '__globals__', 'DREAMER'])
DREAMER Verisinin Tipi: <class 'numpy.ndarray'>
DREAMER Veri Yapısı: [('Data', 'O'), ('EEG_SamplingRate', 'O'), ('ECG_SamplingRate', 'O'), ('EEG_Electrodes', 'O'), ('noOfSubjects', 'O'), ('noOfVideoSequences', 'O'), ('Disclaimer', 'O'), ('Provider', 'O'), ('Version', 'O'), ('Acknowledgement', 'O')]


In [ ]:
print("Örnek valence scores:", labels[:10])
print("Valence score min:", np.min(labels))
print("Valence score max:", np.max(labels))

Örnek valence scores: [0 0 0 0 0 0 0 0 0 0]
Valence score min: 0
Valence score max: 0


In [ ]:
# --- EEG verilerini ön işleme ---
def preprocess_eeg_data(eeg_data, max_length=1000):
    padded_eeg_data = []
    for sample in eeg_data:
        if sample.shape[1] < max_length:
            padding_shape = ((0, 0), (0, max_length - sample.shape[1]))
            padded_sample = np.pad(sample, padding_shape, mode='constant', constant_values=0)
        else:
            padded_sample = sample[:, :max_length]
        padded_eeg_data.append(padded_sample)
    eeg_data = np.array(padded_eeg_data)
    scaler = StandardScaler()
    eeg_data = eeg_data.reshape(eeg_data.shape[0], -1)
    eeg_data = scaler.fit_transform(eeg_data)
    eeg_data = eeg_data.reshape(-1, max_length, 14)
    return eeg_data

In [ ]:
# --- Veri artırma fonksiyonu ---
def augment_eeg_data(eeg_data, labels):
    augmented_data = []
    augmented_labels = []
    for sample, label in zip(eeg_data, labels):
        augmented_data.append(sample)
        augmented_labels.append(label)

        # Gürültü ekleme
        noise = np.random.normal(0, 0.05, sample.shape)
        augmented_data.append(sample + noise)
        augmented_labels.append(label)

        # Zaman kaydırma
        shift = np.roll(sample, shift=5, axis=1)
        augmented_data.append(shift)
        augmented_labels.append(label)

    return np.array(augmented_data), np.array(augmented_labels)

In [ ]:
# --- Veri yükleme ---
file_path = "/content/drive/MyDrive/Colab Notebooks/DREAMER.mat"
eeg_data, labels = load_dreamer_data(file_path)

In [ ]:
# --- Ön işleme ---
eeg_data = preprocess_eeg_data(eeg_data)
labels = LabelEncoder().fit_transform(labels > 5)

In [ ]:
# --- Sınıf dengesini sağlama ---
ros = RandomOverSampler(random_state=42)
eeg_data_flat = eeg_data.reshape(eeg_data.shape[0], -1)
eeg_data_flat, labels = ros.fit_resample(eeg_data_flat, labels)
eeg_data = eeg_data_flat.reshape(-1, 1000, 14)

ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

In [ ]:
# --- Veri artırma ---
X_train, X_test, y_train, y_test = train_test_split(eeg_data, labels, test_size=0.3, random_state=42, stratify=labels)
X_train, y_train = augment_eeg_data(X_train, y_train)

In [ ]:
# --- Model Tanımı ---
model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    GaussianNoise(0.1),
    LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01)),  # Daha fazla birim
    BatchNormalization(),
    Dropout(0.5),
    LSTM(64, return_sequences=False, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

In [ ]:
# --- Callbacks ---
def scheduler(epoch, lr):
    if epoch < 5:
        return 1e-3
    elif epoch < 15:
        return 5e-4
    else:
        return lr * 0.8

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
lr_scheduler = LearningRateScheduler(scheduler)

In [ ]:
# --- Modeli Derleme ---
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# --- Modeli Eğitme ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr, lr_scheduler],
    verbose=1
)

In [ ]:
# --- Eğitim Sonuçlarını Çizdirme ---
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kayıp')
plt.legend()
plt.title("Eğitim ve Doğrulama Kaybı")

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()
plt.title("Eğitim ve Doğrulama Doğruluğu")

plt.tight_layout()
plt.show()

# --- Test Değerlendirmesi ---
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Kayıp: {test_loss:.4f}")
print(f"Test Doğruluk: {test_accuracy:.4f}")